# Game of Life - self-replicating systems with cellular automation
This is a zero player game created by John Horton Conway way back in 1970, where evolution is determined by some initial conditions after which you get to observe how the game evolves.
 
The game world is an infinite grid (in theory) of square 'cells' each of which could be in two possible states either alive or dead (or if you wish populated and unpopulated).

Every cell interacts with its eights neighbors (horizontally, vertically, and diagonally).

There are only 4 rules that govern this interaction and whether a cell emerges from the interaction alive or dead-

1. Any live cell with fewer than two live neighbors dies, as if by underpopulation.
2. Any live cell with two or three live neighbors lives on to the next generation.
3. Any live cell with more than three live neighbors dies, as if by overpopulation.
4. Any dead cell with exactly three live neighbors becomes a live cell, as if by reproduction.

All other cells (which ones do these pertain to by the way) keep their current life status unchanged.

The initial pattern (conditions) of the world grid constitutes the seed of the system, the first generation is created by applying the above rules simultaneously to every cell in this seed. Live, dead, births and deaths occur simultaneously - this happens during the discrete moement called a tick (you'll see what this means in the code and my way of handling it). Afterwhich, the rules continue to be applied repeatedly to create further generations.

Before looking at my implementation below, I encourage you to try to code this on your own from the rules only which is the route I went, throughout the process you'll learn so much and it really is alot of fun. Originally, amazingly, different tile patterns and how they evolve were discovered without computers by using blackboards and graph paper by hand.

It goes without saying that in my code the world is not infiinite - I've made it so my boundary cells (outside_boundary) can either be all dead or all alive (I defaulted to all alive but feel free to change it). 

When I first learned about this automation I was fasicnated and amazed at how such an interesting and dyamic system could be created from random intial conditions four simple rules applied repeatedly. I love the simplicity and beauty of this automation, I hope you will too. Let's get to it-

## Modules and Defaults

In [242]:
#modules required
import numpy as np
import time
#for the visualization below here
import holoviews as hv
import panel as pn
pn.extension(design='material')# this enables panel using the material design/theme
from holoviews.streams import Pipe

#let's set up some defaults for our game, we'll be able to change most of these with panel widgets we'll see at the end
tick_delay = .15 # our tick delay in seconds
world_dim = 60 # our world is square world_dimxworld_dim
tick_life= 100 #how many steps (ticks) to run for
outside_boundary = 1 # cells outside boundary considered what, I kept it at 1 to allow the bounadires to be life giving :) 


## Finding neighbors and applying rules functions

In [243]:
# we need a function to get the neighbors of a cell so that we can pass this to the apply_rules let's implement that
def get_neighbors(cell, world_check):
    #return a lsit of all 8 neighbors any neighbor off the world is classfiied as outside_boundary
    row, col = cell[0], cell[1] #y,x, in numpy arrays I think of it as the row  y and the column x  
    neighbors = [] #list to store the aliveness of our 8 neighbors
    bounding_cells = [[row+r-1,col+c-1] for r in range(3) for c in range(3) if (c*r!=1)] #the c*r!=1 is for that it doesn't count itself (cell) as a bounding
    for bc in bounding_cells: #now let's see if the neighbor lives outside of the world 
        if 0<=bc[0]<world_check.shape[0] and 0<=bc[1]<world_check.shape[1]:# yup you're inside so let's add your aliveness to our list  
            neighbors.append(world_check[bc[0],bc[1]])
        else:
            neighbors.append(outside_boundary) # cells outside the world boundary use our default, other options random, loop around infite? 
    return neighbors 


In [220]:
# first let's write an apply_rules function that enforces the 4 rules of the game, we could have also got fancy and used convulutions :)
def apply_rules(cell, neighbors):
    live_neighbors = neighbors.count(1)
    if cell:  # cell is alive
        if live_neighbors < 2:  # RULE #1 - less than 2 live neighbors then dies :(
            return 0
        elif live_neighbors == 2 or live_neighbors == 3:  #RULE #2 - 2 or 3 live neighbors lives on :)
            return 1
        else:  # RULE #3 >3 live neighbors, dies from over popoulation :(
            return 0
    else:  # cell dead
        if live_neighbors == 3:  #RULE #4 if 3 live neibhors then it becomes alive :)
            return 1
    return 0  # covers any dead cells that don't have 3 live neighbors stay 0 (answer to the question in the description) 


In [228]:
#our main game tick (step) to handle the updating of the board, notice it returns stats and a new world :) 
def game_tick(world_view):
    world_buffer=world_view.copy() # we don't want to modify our world_view becuase we want this to happen all at once using world_view to apply rules to
    stable=True # keep track of if the population (# of 1's) changes or stays the same
    froze=True # keep track of if nothing changes, no deaths or births
    #loop through every cell in our world
    for r in range(world_view.shape[0]):
        for c in range(world_view.shape[1]):
            world_buffer[r,c] = apply_rules(world_view[r,c], get_neighbors((r,c), world_view)) # modify the world buffer according to rules applied on world_view With the neighbors
            froze = (world_buffer[r,c]==world_view[r,c]) and froze# once it goes not stable (change made) stability cannot be restored in this round 
    stable = (np.count_nonzero(world_view)==np.count_nonzero(world_buffer))#stable if the nonzero before and after ==    
    return (world_buffer,f"** population {np.count_nonzero(world_view)} ** stable {stable} ** Frozen {froze}") #return the modified world_buffer with stats
  

In [229]:
#init the world with dimension dimxdim, random generated or not and points if not
def init_world(dim, random=True,points=None):
    if random:
        return np.random.randint(0,2,size=(dim,dim))
    else:
        create_world = np.zeros((dim,dim), dtype=np.byte)
        if points is not None:
            for p in points: #if we pass intial points to try some inital conditions ourselves 
                if 0<=p[0]<create_world.shape[0] and 0<=p[1]<create_world.shape[1]:# make sure we're inside the world before we try to make alive  
                    create_world[p[0],p[1]]=1 
        return create_world #return our new world



## Now let's put it all together in a game loop, visualization with panel and holoviews. You can move and zoom the world around, very slick. 

In [244]:
# Create a Pipe stream to callback with the data to the holoview dynamic map Image, real time view
pipe = Pipe(np.zeros((400,400), dtype=np.byte))
#create a dynamic image representing our world you can experiment here with different color maps
image = hv.DynamicMap(hv.Image, streams=[pipe]).opts(
    width=400, height=400, xaxis=None, yaxis=None, toolbar=None,aspect='equal',cmap='viridis')

#now let's make our widgets so we can fiddle with defaults for our world
tick_delay_slider = pn.widgets.FloatSlider(name='tick_delay',start=.01, end=1,step=.01,value=tick_delay)
world_dim_slider = pn.widgets.IntSlider(name='world_size',start=10,end=150,step=2,value=world_dim)
tick_life_slider = pn.widgets.IntSlider(name='world_life',start=20,end=5000,step=10,value=tick_life)
static_text = pn.widgets.StaticText(name='World Status', value='')
str_pane1 = pn.pane.Str('random world?')
random_switch = pn.widgets.Switch(name='Switch1', value=True)
str_pane2 = pn.pane.Str('boundaries alive?')
boundary_switch = pn.widgets.Switch(name='Switch2', value=True)
                        

#our main game loop where rules are applied and new world generations computed 
def game_loop(event):
    # make an intial world
    world = init_world(world_dim_slider.value,random=random_switch.value,points=None) 
    global outside_boundary
    outside_boundary=boundary_switch.value
    for i in range(tick_life_slider.value+1): #our main game loop       
        #data=np.random.randint(0,2,size=(world_dim_slider.value,world_dim_slider.value))
        (world,status) = game_tick(world) 
        #send another pipe stream to a text box or some other way just using panel?
        pipe.send(world)
        static_text.value=f"tick {i}/{tick_life_slider.value} {status}"
        time.sleep(tick_delay_slider.value)


button = pn.widgets.Button(name='Go')
button.on_click(game_loop) # link up our button to trigger the game_loop function we made

# put it all together to show to the world :) 
panel = pn.Column(pn.Row(world_dim_slider,tick_life_slider,tick_delay_slider),pn.Row(str_pane1,random_switch,str_pane2,boundary_switch),
                  pn.Row(button),pn.Row(static_text), image)
panel.servable() # (this creates a default layout as we) and shows everything ready to go!

Column(design=<class 'panel.theme.materi...)
    [0] Row(design=<class 'panel.theme.materi...)
        [0] IntSlider(design=<class 'panel.theme.materi..., end=150, name='world_size', start=10, step=2, value=60)
        [1] IntSlider(design=<class 'panel.theme.materi..., end=5000, name='world_life', start=20, step=10, value=100)
        [2] FloatSlider(design=<class 'panel.theme.materi..., name='tick_delay', start=0.01, step=0.01, value=0.15)
    [1] Row(design=<class 'panel.theme.materi...)
        [0] Str(str, design=<class 'panel.theme.materi...)
        [1] Switch(design=<class 'panel.theme.materi..., value=True)
        [2] Str(str, design=<class 'panel.theme.materi...)
        [3] Switch(design=<class 'panel.theme.materi..., value=True)
    [2] Row(design=<class 'panel.theme.materi...)
        [0] Button(design=<class 'panel.theme.materi..., name='Go')
    [3] Row(design=<class 'panel.theme.materi...)
        [0] StaticText(design=<class 'panel.theme.materi..., name='World Status')
    [4] HoloViews(DynamicMap, design=<class 'panel.theme.materi..., height=400, sizing_mode='fixed', width=400)

## I had a such a great time making this and spending way too long watching the 'space ships' fly around. Try turning off random and watch the boundary cells generate symmetrically, incredible fractal kaleidascope-like patterns emerge. 
## Maybe you noticed the code I have in init_world for the creation of intial conditions with zero filled world, go ahead and implement that if you'd like and let me know how it goes. This notebook is on my github for this interested in downloading.

## I hope you had fun and maybe it inspired you to make your own automation. Happy coding!!